In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn.functional import relu
from scipy.io import loadmat
import os
from os import listdir
import pandas as pd
from skimage import io
from PIL import Image
from scipy.io import loadmat

window = 15

# Goose 1st series: 180322-180401
# save_path = '/mnt/pesaranlab/People/Capstone_students/Yue/data_Goose_1st/'
# nights = ['180322', '180323', '180324', '180325', '180326', '180327', '180328', '180329', '180330', '180331', '180401']
# recs = ['001','002','003','004','005','006','007','008','009']

# Goose 2nd series: 180409-180414
save_path = '/mnt/pesaranlab/People/Capstone_students/Daisy/data_15sW/'
nights = ['180322','180323','180324','180325','180326','180327','180329','180330','180331','180401','180409','180410','180411','180412','180413','180414']
recs = ['001','002','003','004','005','006','007','008','009']

In [3]:
for night in nights:
    for rec in recs:
        print("Checking ", night, ' ', rec)

        try: 
            NSV = loadmat('/vol/sas2b/Goose_Multiscale_M1_Wireless/'+night+'/'+rec+'/rec'+rec+'.NightStateVars.mat')
            movement_states = NSV['UserData'][0][0]['MovementStates_wrt_spec'][0] 
            # tuples indicating start and end of movement
            sleep_states = NSV['UserData'][0][0]['SleepStates_wrt_spec'][0]
            # tuples indicating start and end of sleep
            # not necessarily overlap due to out-of-camera issue
            spec_data = loadmat('/mnt/pesaranlab/People/Capstone_students/Spectrogram_mat_data/Goose/N10W1dn1_'
                                +night+'_rec'+rec+'.mat')
            times = spec_data['Spec_per_Ch'][0,0]['Ch1'][0,0]['ti'][0][:] // 1000
            specs = []
            all_badtimes = np.array([])

            for ch in range(1,63):
                ztotSpec = spec_data['Spec_per_Ch']['Ch'+str(ch)][0][0]['ztotSpec'][0][0]
                specs.append(ztotSpec)
                badtimes = spec_data['Spec_per_Ch']['Ch'+str(ch)][0][0]['badtimes_2'][0][0]
                all_badtimes = np.concatenate([all_badtimes, badtimes.flatten()])
    
            
            ztotSpecs = np.stack(specs)
            all_badtimes = np.array(list(set(all_badtimes)))
            print('zts.shape=',ztotSpecs.shape)
            
            movement_states_idx = []
            for state in movement_states:
                start = state[0][0]
                start_idx =  (np.abs(times-start)).argmin()
                stop = state[0][1]
                stop_idx = (np.abs(times-stop)).argmin()
                ###
                if (times-start)[start_idx] < 0:
                    start_idx += 1
                if (times-stop)[stop_idx] > 0:
                    stop_idx -= 1
                ###
                movement_states_idx.append([start_idx, stop_idx])
                
            sleep_states_idx = []
            for state in sleep_states:
                start = state[0][0]
                start_idx =  (np.abs(times-start)).argmin()
                stop = state[0][1]
                stop_idx = (np.abs(times-stop)).argmin()
                ###
                if (times-start)[start_idx] < 0:
                    start_idx += 1
                if (times-stop)[stop_idx] > 0:
                    stop_idx -= 1
                ###
                sleep_states_idx.append([start_idx, stop_idx])
            
            badtime_idx = []
            for badtime in all_badtimes:
                idx = (np.abs(times-badtime)).argmin()
                badtime_idx.append(idx)

            # for all bad times, imput NaN 
            for idx in badtime_idx:
                 ztotSpecs[:,idx,:] = np.nan
    
    
            # saving
            for s in sleep_states_idx:
                start = s[0]
                stop = s[1]
                if stop-start < window:
                    continue

                sspec = torch.from_numpy(ztotSpecs[:,start:stop,:])
                sspecs = torch.split(sspec,window,dim=1)

                stime = times[start:stop]
                if sspecs[-1].shape[1] != window:
                    stimes = np.array(np.split(stime[:-(sspecs[-1].shape[1])], len(sspecs)-1))
                else:
                    stimes = np.array(np.split(stime, len(sspecs)))
                stimes_mid = stimes[:, window//2]
                
                for i, arr in enumerate(sspecs):
                    if not torch.isnan(arr.sum()) and arr.shape[1] == window:
                        np.save(save_path+'sleep/'+night+'_'+rec+'_'+str(ch)+'_time'+str(stimes_mid[i])+'_sleep.npy',arr.numpy())
   
            for s in movement_states_idx:
                start = s[0]
                stop = s[1]
                if stop-start < window:
                    continue

                sspec = torch.from_numpy(ztotSpecs[:,start:stop,:])
                sspecs = torch.split(sspec,window,dim=1)
                
                stime = times[start:stop]
                if sspecs[-1].shape[1] != window:
                    stimes = np.array(np.split(stime[:-(sspecs[-1].shape[1])], len(sspecs)-1))
                else:
                    stimes = np.array(np.split(stime, len(sspecs)))
                stimes_mid = stimes[:, window//2]

                for i, arr in enumerate(sspecs):
                    if not torch.isnan(arr.sum()) and arr.shape[1] == window:
                        np.save(save_path+'move/'+night+'_'+rec+'_'+str(ch)+'_time'+str(stimes_mid[i])+'_move.npy',arr.numpy())
   

        except Exception as e:
            print(e)
        
total_nonmvmt_samples = len(os.listdir(save_path + 'sleep')) 
total_mvmt_samples = len(os.listdir(save_path + 'move')) 
        
print("Done! Total non-movement = ", total_nonmvmt_samples, ' and total movement = ', total_mvmt_samples)

Checking  180322   001
zts.shape= (62, 7194, 100)
Checking  180322   002
zts.shape= (62, 7191, 100)
Checking  180322   003
zts.shape= (62, 7191, 100)
Checking  180322   004
zts.shape= (62, 7191, 100)
Checking  180322   005
zts.shape= (62, 7191, 100)
Checking  180322   006
zts.shape= (62, 7191, 100)
Checking  180322   007
zts.shape= (62, 5633, 100)
Checking  180322   008
index 0 is out of bounds for axis 0 with size 0
Checking  180322   009
index 0 is out of bounds for axis 0 with size 0
Checking  180323   001
zts.shape= (62, 7190, 100)
Checking  180323   002
zts.shape= (62, 7191, 100)
Checking  180323   003
zts.shape= (62, 7196, 100)
Checking  180323   004
zts.shape= (62, 7197, 100)
Checking  180323   005
zts.shape= (62, 7197, 100)
Checking  180323   006
zts.shape= (62, 7197, 100)
Checking  180323   007
zts.shape= (62, 7197, 100)
Checking  180323   008
index 0 is out of bounds for axis 0 with size 0
Checking  180323   009
index 0 is out of bounds for axis 0 with size 0
Checking  180324

zts.shape= (62, 7191, 100)
Checking  180412   002
zts.shape= (62, 7191, 100)
Checking  180412   003
zts.shape= (62, 7187, 100)
Checking  180412   004
zts.shape= (62, 7190, 100)
Checking  180412   005
zts.shape= (62, 7191, 100)
Checking  180412   006
zts.shape= (62, 6670, 100)
Checking  180412   007
[Errno 2] No such file or directory: '/vol/sas2b/Goose_Multiscale_M1_Wireless/180412/007/rec007.NightStateVars.mat'
Checking  180412   008
[Errno 2] No such file or directory: '/vol/sas2b/Goose_Multiscale_M1_Wireless/180412/008/rec008.NightStateVars.mat'
Checking  180412   009
[Errno 2] No such file or directory: '/vol/sas2b/Goose_Multiscale_M1_Wireless/180412/009/rec009.NightStateVars.mat'
Checking  180413   001
zts.shape= (62, 7191, 100)
Checking  180413   002
zts.shape= (62, 7191, 100)
Checking  180413   003
zts.shape= (62, 7196, 100)
Checking  180413   004
zts.shape= (62, 7197, 100)
Checking  180413   005
zts.shape= (62, 7197, 100)
Checking  180413   006
zts.shape= (62, 7197, 100)
Checki

In [4]:
night, rec = '180331', '008' # index 0 is out of bounds for axis 0 with size 0

In [5]:
NSV = loadmat('/vol/sas2b/Goose_Multiscale_M1_Wireless/'+night+'/'+rec+'/rec'+rec+'.NightStateVars.mat')
NSV['UserData'][0][0]['SleepStates_wrt_spec'] # this recording has no sleep stage

array([], shape=(0, 0), dtype=object)